In [1]:
from setup_notebook import *
import os

# Introduction

## Load study files

In [2]:
study_name = 'ERFCVU'

In [3]:
files = filter(lambda e: study_name in e, os.listdir(setup.path_raw_data))
files = list(files)
files

['.~lock.ERFCVU_EXKDCU_FINAL IMC EVD Codebook 5.9.2016_20190610.xlsx#',
 '.~lock.ERFCVU_PSSPatientBackground2019_June.csv#',
 'ERFCVU_EVDLabData_2019_November.csv',
 'ERFCVU_EVDPatientData_2019_November.csv',
 'ERFCVU_EVDRoundingData_2019_November.csv',
 'ERFCVU_EVDTreatmentData_2019_November.csv',
 'ERFCVU_EVDTriageData_2019_November.csv',
 'ERFCVU_EXKDCU_FINAL IMC EVD Codebook 5.9.2016_20190610.xlsx',
 'ERFCVU_PSSExit2019_June.csv',
 'ERFCVU_PSSFamilyandCaregiverSupport2019_June.csv',
 'ERFCVU_PSSPatientandFamilyFollowUp2019_June.csv',
 'ERFCVU_PSSPatientBackground2019_June.csv',
 'ERFCVU_EVDDischargeData_2019_November.csv']

In [4]:
for f in files:
    fp = join(setup.path_raw_data, f)
    print('------')
    !wc -l "$fp"
    !head "$fp" -n 2
    print()

------
0 ../IDDO_EDP/EDP_dataset/Original dataset/.~lock.ERFCVU_EXKDCU_FINAL IMC EVD Codebook 5.9.2016_20190610.xlsx#
,maousi,maousi-Z390-GAMING-X,23.11.2020 23:06,file:///home/maousi/.config/libreoffice/4;
------
0 ../IDDO_EDP/EDP_dataset/Original dataset/.~lock.ERFCVU_PSSPatientBackground2019_June.csv#
,maousi,maousi-Z390-GAMING-X,23.11.2020 23:30,file:///home/maousi/.config/libreoffice/4;
------
4298 ../IDDO_EDP/EDP_dataset/Original dataset/ERFCVU_EVDLabData_2019_November.csv
PatientKey,ETUKey,AGE,GENDER,Region,SubRegion,WardNumber,BedNumber,DateBloodDraw,TimeBloodDraw,LabCompletingTest,TestDate,Result,ResultDate,SampleNumber,LaboratoryCode,Referrer,SpecimenCollectionDate,DateSpecimenReceivedByLab,SpecimenType,InitialRepeat,HospitalizationDate,EbolaPCRResult,CT,CTqRT_PCRI,CT_ZAIRE,CT_MGB,MalariaResult,DateOnset,ETC,Comments,PatientStatus,MinimumRequirementStatus,EtuName,Admitkey,Flag
24515,2,NULL,U,Port Loko,Koya,NULL,NULL,NULL,NULL,PHE - Kerry Town,NULL,NULL,NULL,NULL,NULL,NULL,26/

**The `.xlsx` file is a data dictionnary**. We load all `.csv` files:

In [5]:
csv = {}
for f in files:
    if not f.endswith('.csv'):
        continue
        
    fname = f.split('_')[1]
    print(fname, end=', ')
    csv[fname] = pd.read_csv(join(setup.path_raw_data, f))

EVDLabData, EVDPatientData, EVDRoundingData, EVDTreatmentData, 

/home/maousi/anaconda3/envs/ml/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (16,17,18,19,20,23,49,50,51,52,53,54) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/home/maousi/anaconda3/envs/ml/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (0,4,7,8,9,10,12,14,16,18,20,22,24,26,27,28,29,30,31,32,33,34,35,37,39,41,42,43,44,45,46,47,49,50,51,52,53,54,55,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,73,74,75,76,77,80,83,85,86,87,88,89,90,91,92,93,94,95,96,97) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


EVDTriageData, PSSExit2019, PSSFamilyandCaregiverSupport2019, PSSPatientandFamilyFollowUp2019, PSSPatientBackground2019, EVDDischargeData, 

In [6]:
# Describe dataframes:
for name, df in csv.items():
    npatients = 'no patients'
    if 'PatientKey' in df.columns:
        npatients = f'{df.PatientKey.nunique()} unique patients'
        
    print(f'{name}: shape {df.shape}, {npatients}')

EVDLabData: shape (4297, 36), 2101 unique patients
EVDPatientData: shape (2739, 13), 2739 unique patients
EVDRoundingData: shape (16030, 66), 2070 unique patients
EVDTreatmentData: shape (25003, 100), 2518 unique patients
EVDTriageData: shape (2744, 91), 2715 unique patients
PSSExit2019: shape (1663, 22), 1644 unique patients
PSSFamilyandCaregiverSupport2019: shape (4913, 11), 1151 unique patients
PSSPatientandFamilyFollowUp2019: shape (11343, 9), 1026 unique patients
PSSPatientBackground2019: shape (1669, 16), no patients
EVDDischargeData: shape (2676, 25), 2647 unique patients


In [7]:
from collections import Counter

In [8]:
# ------ Report columns
cols = []
for df in csv.values():
    cols.extend(df.columns)
column_count = Counter(cols)

# Show counts of columns across all dataframe
sorted(list(column_count.items()), key=lambda tpl: tpl[1], reverse=True)[:10]

[('PatientKey', 9),
 ('AdmitKey', 6),
 ('ETUKey', 5),
 ('Flag', 5),
 ('ETU Key', 3),
 ('Audit Source Field', 3),
 ('EtuName', 2),
 ('Admitkey', 2),
 ('NameOfLab', 2),
 ('Headache', 2)]

## Data dictionnary

In [9]:
dic_path = join(setup.path_raw_data, 'ERFCVU_EXKDCU_FINAL IMC EVD Codebook 5.9.2016_20190610.xlsx')
dic_file = pd.ExcelFile(dic_path)
sheets = dic_file.sheet_names
sheets

['Patient',
 'Discharge',
 'Triage',
 'Lab',
 'Rounding',
 'Treatment',
 'PSS Pt Background',
 'PSS Fam & Caregiver Support',
 'PSS Exit',
 'PSS Pt & Fam Follow Up',
 'PSS Pt Interventions',
 'PSS Pt Interventions Aggr']

In [10]:
dic = {
    sheet: pd.read_excel(dic_file, sheet_name=sheet, header=2)
    for sheet in sheets
}

# Lab data

In [11]:
lab = csv['EVDLabData']
lab.isna().mean().sort_values()

PatientKey                   0.000000
ETUKey                       0.000000
Admitkey                     0.000000
LabCompletingTest            0.018850
GENDER                       0.030719
SpecimenCollectionDate       0.031650
EbolaPCRResult               0.032814
InitialRepeat                0.037003
Region                       0.156854
Referrer                     0.250175
MalariaResult                0.340005
DateOnset                    0.387014
SubRegion                    0.389109
CT                           0.646032
EtuName                      0.669770
TestDate                     0.687224
LaboratoryCode               0.688620
SpecimenType                 0.699791
AGE                          0.710263
DateBloodDraw                0.740982
MinimumRequirementStatus     0.786828
PatientStatus                0.787061
SampleNumber                 0.841285
WardNumber                   0.899697
ResultDate                   0.899697
Result                       0.899697
BedNumber   

In [12]:
lab

,PatientKey,ETUKey,AGE,GENDER,Region,SubRegion,WardNumber,BedNumber,DateBloodDraw,TimeBloodDraw,...,CT_MGB,MalariaResult,DateOnset,ETC,Comments,PatientStatus,MinimumRequirementStatus,EtuName,Admitkey,Flag
0,24515,2,NaN,U,Port Loko,Koya,NaN,NaN,NaN,NaN,...,NaN,NaN,22/11/2014,NaN,NaN,NaN,NaN,NaN,1,NaN
1,24516,2,NaN,U,Port Loko,Maforki,NaN,NaN,NaN,NaN,...,NaN,NaN,22/11/2014,NaN,NaN,NaN,NaN,NaN,1,NaN
2,24561,2,NaN,U,Port Loko,Bureh Kasseh Ma,NaN,NaN,NaN,NaN,...,NaN,NaN,21/11/2014,NaN,NaN,NaN,NaN,NaN,1,NaN
3,24523,2,NaN,U,Port Loko,Koya,NaN,NaN,NaN,NaN,...,NaN,NaN,18/11/2014,NaN,NaN,NaN,NaN,NaN,1,NaN
4,24514,2,NaN,U,Port Loko,Lokomasama,NaN,NaN,NaN,NaN,...,NaN,NaN,24/11/2014,NaN,NaN,NaN,NaN,NaN,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4292,25053,5,NaN,F,Western_Urban,NaN,NaN,NaN,NaN,NaN,...,NaN,Positive,25/04/2015,NaN,NaN,NaN,NaN,NaN,1,NaN
4293,25054,5,NaN,M,Bombali,Bombali Sebora,NaN,NaN,NaN,NaN,...,NaN,Positive,23/04/2015,NaN,NaN,NaN,NaN,NaN,1,NaN
4294,24222,2,NaN,F,Port_Loko,Marampa,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
4295,24279,2,NaN,M,Port_Loko,Marampa,NaN,NaN,NaN,NaN,...,NaN,Negative,09/04/2015,NaN,NaN,NaN,NaN,NaN,1,NaN


In [13]:
for col in lab.columns[1:]:
    print(col, ':')
    print(lab[col].value_counts(), '\n')

ETUKey :
5    1817
2    1103
1     701
4     461
3     215
Name: ETUKey, dtype: int64 

AGE :
25.0    49
30.0    43
32.0    42
40.0    41
3.0     41
        ..
69.0     1
83.0     1
74.0     1
76.0     1
90.0     1
Name: AGE, Length: 79, dtype: int64 

GENDER :
M          2027
F          1896
U           240
Unknown       1
FSB           1
Name: GENDER, dtype: int64 

Region :
Bombali               1015
Port_Loko              582
Bong                   557
Port Loko              374
Kambia                 331
Margibi                274
Tonkolili              150
NOT PRESENT            116
Grand Bassa             61
NOT STATED              47
Western_Urban           22
Montserrado             19
Moyamba                 14
Western_Rural           11
BOMBALI                  8
Koinadugu                6
Kono                     5
Kenema                   5
Not stated               4
Gbarpolu                 3
not stated               3
Bo                       2
NOT LEGIBLE              2

In [14]:
lab.PatientKey.nunique()

2101

## Dates

In [15]:
# Columns containing "date" (case insensitive)
dates = list(filter(lambda e: 'date' in e.lower(), lab.columns))
dates

['DateBloodDraw',
 'TestDate',
 'ResultDate',
 'SpecimenCollectionDate',
 'DateSpecimenReceivedByLab',
 'HospitalizationDate',
 'DateOnset']

In [16]:
# Fraction of missing values in date columns
lab[dates].isna().mean().sort_values()

SpecimenCollectionDate       0.031650
DateOnset                    0.387014
TestDate                     0.687224
DateBloodDraw                0.740982
ResultDate                   0.899697
DateSpecimenReceivedByLab    0.912963
HospitalizationDate          0.997207
dtype: float64

We compare 

In [17]:
lab[~lab.DateOnset.isna()][['DateOnset', 'SpecimenCollectionDate']]

,DateOnset,SpecimenCollectionDate
0,22/11/2014,26/11/2014
1,22/11/2014,26/11/2014
2,21/11/2014,26/11/2014
3,18/11/2014,28/11/2014
4,24/11/2014,29/11/2014
...,...,...
4289,23/04/2015,26/04/2015
4291,23/04/2015,26/04/2015
4292,25/04/2015,26/04/2015
4293,23/04/2015,26/04/2015


In [18]:
lab[~lab.TestDate.isna()][['TestDate', 'DateOnset', 'SpecimenCollectionDate']]

,TestDate,DateOnset,SpecimenCollectionDate
137,20/10/2014,17/10/2014,21/10/2014
159,13/11/2014,08/11/2014,12/11/2014
160,13/11/2014,11/11/2014,12/11/2014
161,26/11/2014,11/11/2014,26/11/2014
162,28/11/2014,11/11/2014,28/11/2014
...,...,...,...
4262,16/11/2014,14/11/2014,16/11/2014
4263,17/11/2014,01/11/2014,17/11/2014
4264,06/09/2015,04/06/2015,06/09/2015
4281,28/12/2014,16/12/2014,28/12/2014


**TODO: what to do with dates, are they reliable? what is `SpecimenCollectionDate`?**

## Patients overview

We describe the distribution of the number of records per patient:

In [19]:
lab.PatientKey.value_counts().describe()

count    2101.000000
mean        2.045217
std         1.219427
min         1.000000
25%         1.000000
50%         2.000000
75%         2.000000
max        14.000000
Name: PatientKey, dtype: float64

In [20]:
# Records of patient with 14 records
weird_patient = lab.PatientKey.value_counts().index[0]
lab[lab.PatientKey == weird_patient][['EbolaPCRResult'] + dates]

,EbolaPCRResult,DateBloodDraw,TestDate,ResultDate,SpecimenCollectionDate,DateSpecimenReceivedByLab,HospitalizationDate,DateOnset
3345,Negative,NaN,NaN,NaN,06/04/2015,NaN,NaN,NaN
3346,Negative,NaN,NaN,NaN,06/04/2015,NaN,NaN,NaN
3398,Negative,NaN,NaN,NaN,16/03/2015,NaN,NaN,16/03/2015
3399,Negative,NaN,NaN,NaN,16/03/2015,NaN,NaN,16/03/2015
3490,Negative,NaN,NaN,NaN,18/03/2015,NaN,NaN,13/03/2015
3491,Negative,NaN,NaN,NaN,18/03/2015,NaN,NaN,13/03/2015
3609,POSITIVE,NaN,NaN,NaN,24/03/2015,NaN,NaN,NaN
3610,POSITIVE,NaN,NaN,NaN,24/03/2015,NaN,NaN,NaN
3810,POSITIVE,NaN,NaN,NaN,08/04/2015,NaN,NaN,NaN
3811,POSITIVE,NaN,NaN,NaN,08/04/2015,NaN,NaN,NaN


## Ebola Target

In [21]:
# Harmonize
lab.EbolaPCRResult = lab.EbolaPCRResult.str.lower()

The following `Series` contains the number of unique patients for each category of `EbolaPCRResult`:

In [22]:
lab.groupby('EbolaPCRResult').PatientKey.nunique().sort_values()

EbolaPCRResult
not done, specimen insufficient                                          1
specimen repeatedly inderterminate, please send a repeat specimen.       1
unknown                                                                  1
insufficient sample                                                      2
sample degraded - please repeat sampling                                 2
pending                                                                  4
rejected (see comments)                                                 10
indeterminate                                                           17
lab to repeat test - report to follow                                   18
indeterminate (see comments)                                            21
pending (see comments)                                                  30
positive                                                               425
negative                                                              1841
Name: Pati

Note that this sum to more than the total number of patients, since a patient can appear in several categories. Let's drop the rows with `EbolaPCRResult` that is neither `positive` or `negative` (therefore, `nan`s are also discarded): 

In [23]:
mask_valid_target = lab.EbolaPCRResult.isin(['positive', 'negative'])
print(mask_valid_target.value_counts())

# Remove unvalid targets
lab = lab[mask_valid_target]

True     4030
False     267
Name: EbolaPCRResult, dtype: int64


The following `Series` computes the number of distinct ebola target per patient, i.e. whether a patient is either positive or negative, or has both targets:

In [24]:
target_count_per_patient = lab.groupby('PatientKey').EbolaPCRResult.nunique()
target_count_per_patient

PatientKey
1        1
2        1
3        1
4        1
5        1
        ..
26121    1
26122    1
26123    1
26138    1
26160    1
Name: EbolaPCRResult, Length: 2056, dtype: int64

We want to know how many patients have both positive and negative targets:

In [25]:
target_count_per_patient.value_counts()

1    1846
2     210
Name: EbolaPCRResult, dtype: int64

We should check the consistency of records that have both positive and negative records. The problem is that the dates seem inconsistent, so we cannot use dates to check chronology of records.

**TODO: find better result than dropping those records.**

**TODO: check if there are many Ebola positive records within patients with two records.**

In [26]:
patients_single_target = target_count_per_patient[target_count_per_patient == 1].index
patients_single_target.size

1846

In [27]:
print(lab.PatientKey.nunique())
# Drop patients having more than one target
lab = lab[lab.PatientKey.isin(patients_single_target)]
lab.PatientKey.nunique()

2056


1846

Now, all records refer to patients having a single target (but possibly several records of the same target).

In [28]:
# Recover the (now unique) ebola test result of each patient
target = lab.groupby('PatientKey').EbolaPCRResult.unique().apply(lambda lst: lst[0])
target.value_counts()

negative    1631
positive     215
Name: EbolaPCRResult, dtype: int64

In [29]:
# Convert to binary
target = (target == 'positive') * 1
target.value_counts()

0    1631
1     215
Name: EbolaPCRResult, dtype: int64

**We now have the classifier target specifying whether a patient has ebola or not. This is a `pandas.Series`: the index is `PatientKey`, the value is binary.**

## Malaria test

In [30]:
lab.MalariaResult.value_counts()

Negative                         1097
Not Applicable                    544
Positive                          457
Not Done                          174
NEGATIVE                           21
POSITIVE                           12
NOT DONE                            7
NOT APPLICABLE                      3
Not done                            1
POSITIVE T1, WEAK POSITIVE T2       1
Test not requested                  1
insufficient                        1
negative                            1
POSITIVE T1 & T2                    1
Name: MalariaResult, dtype: int64

**TODO: clean malaria**